In [3]:
import pandas as pd
from pathlib import Path
import numpy as np
import os

In [4]:
contaminants_data = []

contaminants_list = ["2,3,7,8-Tetrachlorodibenzo-p-dioxin","Dieldrin","Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)","Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)","Mercury","Lead","Cyanide"]
clean_data_dir = '../cleandata'

for filename in os.listdir(clean_data_dir):
    if filename.endswith(".csv"):
        csv_data = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True)
        for item in contaminants_list:
            contaminants_filtered = csv_data[csv_data['CHEMICAL_NAME'] == item]
            contaminants_filtered = contaminants_filtered.iloc[: , 1:]
            contaminants_data.append(contaminants_filtered)

contaminants_df = pd.concat(contaminants_data)

In [6]:

contaminants_df.to_csv('../../notebooks/chemical_filter.csv')





